In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import sys
 
img = cv2.imread('4.2.07.tiff', 0)  
plt.imshow(img, cmap='gray'); 
plt.axis('off')
plt.title('Original Image')
plt.show()
blur = cv2.GaussianBlur(img, (5,5), 1)
log = cv2.Laplacian(blur, cv2.CV_64F)
blur1 = cv2.GaussianBlur(img, (5,5), 1)
blur2 = cv2.GaussianBlur(img, (5,5), 2)
dog = blur1 - blur2
 
filtered = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)

plt.figure(figsize=(12,4))
 
plt.subplot(1,3,2); plt.imshow(log, cmap="gray"); plt.title("LoG")
plt.subplot(1,3,3); plt.imshow(dog, cmap="gray"); plt.title("DoG")
plt.subplot(1,3,1); plt.imshow(log, cmap="gray"); plt.title("bilateral")
plt.show()

loG: softening a photo first, then tracing the outlines with a pencil.
DoG: Taking two photos with different levels of blur and subracting them so only the edges stand out
Bilateral: like applying a beauty filter that smooths skin but keeps the face shape clear.

In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import sys
 
img = cv2.imread('4.2.07.tiff',cv2.IMREAD_GRAYSCALE)
sift_Detector = cv2.SIFT_create()
sift_keypoints, descriptors = sift_Detector.detectAndCompute(img.astype(np.uint8), None)
img_sift = cv2.drawKeypoints(img, sift_keypoints,None)
cv2.imshow("sift keypoints",img_sift)
cv2.waitKey(0)
ORB_object = cv2.ORB_create()
keypoints = ORB_object.detect(img)
keypoints, descriptors = ORB_object.compute(img, keypoints)
imageresult = cv2.drawKeypoints(img, keypoints, None, color=(255,0,0), flags=0)
cv2.imshow('ORB_image', imageresult)
cv2.waitKey(0)
orb = cv2.ORB_create()
img_keypoints, img_descriptors = orb.detectAndCompute(img,None) 
 
cv2.destroyAllWindows()
print("Number of SIFT keypoints:", len(sift_keypoints))
print("Number of ORB keypoints:", len(keypoints))


SIFT number:720
ORB number:500
SIFT uncovers nearly two and a half times more interest points than ORB’s default setup.  
ORB’s preset nfeatures=500 imposes an upper bound on detected points unless you tweak that limit.  
SIFT relies on a scale-space Difference-of-Gaussian approach for richer detail extraction, while ORB opts for speed and rotational robustness over sheer point count.

In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import sys
 
img1 = cv2.imread('4.2.07.tiff',cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('4.2.06.tiff',cv2.IMREAD_GRAYSCALE)
sift_Detector = cv2.SIFT_create()
img1_keypoints, img1_descriptors = sift_Detector.detectAndCompute(img1, None)
img2_keypoints, img2_descriptors = sift_Detector.detectAndCompute(img2, None)
bf = cv2.BFMatcher()
matches = bf.knnMatch(img1_descriptors, img2_descriptors, k=2)
number_of_matches = len(matches)
img_matches = cv2.drawMatchesKnn(img1, img1_keypoints, img2, img2_keypoints, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.figure(figsize=(12, 6))
plt.imshow(img_matches)
plt.axis('off')
plt.title('SIFT Matches ({number_of_matches} matches)')
plt.show()
 

Most feature matches look correct, but a few mismatches are present. Using FLANN helps improve the matching process by finding closer matches efficiently, and applying a ratio test can further reduce errors.

In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import sys
import skimage.util 
from skimage.util  import random_noise
img = cv2.imread('4.2.07.tiff',cv2.IMREAD_GRAYSCALE)
# Add 'salt and pepper' noise to the image with density of 0.02
noisyImg = random_noise(img, mode="s&p",amount=0.02)
noisyImg = np.array(255*noisyImg, dtype="uint8")
plt.figure(), plt.imshow(noisyImg, cmap="gray", vmin=0, vmax=255),plt.axis('off'), plt.title('Noisy Image')
kernel= np.array([[-1,5,-1],[0,-1,0]])
sharpened= cv2.filter2D(img,-1,kernel)

blur1 = cv2.GaussianBlur(img, (5,5), 1)
blur2 = cv2.GaussianBlur(img, (5,5), 2)
dog = blur1 - blur2
plt.figure(figsize=(12,4))
plt.subplot(1,3,3); plt.imshow(dog, cmap="gray"); plt.title("DoG")

plt.show()

When I added noise, the edges became messy and harder to match. Sharpening, on the other hand, made the features stand out more, so matching was easier. Noise hurts detection, sharpening helps it.